In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
try:
    import deepmatcher
except:
    !pip install -qqq deepmatcher
import deepmatcher  as dm 
import torch
torch.cuda.is_available()
import json
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 51kB 3.8MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 296kB 14.9MB/s 
  ERROR: Failed building wheel for fasttextmirror
    Running setup.py install for fasttextmirror ... done


In [ ]:
df1 = pd.read_csv("/content/drive/My Drive/AbtBuyData/tableA.csv")
df2 = pd.read_csv("/content/drive/My Drive/AbtBuyData/tableB.csv") 
df = pd.DataFrame(columns = ['pair_id', 'label','left_name', 'left_description', 'left_price','right_name','right_description','right_price'])
dfTr=pd.read_csv("/content/drive/My Drive/AbtBuyData/train.csv")
dfTs=pd.read_csv("/content/drive/My Drive/AbtBuyData/test.csv")
dfVl=pd.read_csv("/content/drive/My Drive/AbtBuyData/valid.csv")

dfTrNew = pd.DataFrame(columns = ['pair_id', 'label','left_name', 'left_description', 'left_price','right_name','right_description','right_price'])
for ind in dfTr.index:
  pair_id=str(dfTr['ltable_id'][ind])+"#"+str(dfTr["rtable_id"][ind])
  dfTrNew = dfTrNew.append({'pair_id': pair_id, 'label': dfTr['label'][ind], 'left_name': df1['name'][dfTr['ltable_id'][ind]],'left_description':df1['description'][dfTr['ltable_id'][ind]],'left_price':df1['price'][dfTr['ltable_id'][ind]],'right_name':df2['name'][dfTr['rtable_id'][ind]],'right_description':df2['description'][dfTr['rtable_id'][ind]],'right_price':df2['price'][dfTr['rtable_id'][ind]]}, ignore_index=True)

dfTsNew = pd.DataFrame(columns = ['pair_id', 'label','left_name', 'left_description', 'left_price','right_name','right_description','right_price'])
for ind in dfTs.index:
  pair_id=str(dfTs['ltable_id'][ind])+"#"+str(dfTs["rtable_id"][ind])
  dfTsNew = dfTsNew.append({'pair_id': pair_id, 'label': dfTs['label'][ind], 'left_name': df1['name'][dfTs['ltable_id'][ind]],'left_description':df1['description'][dfTs['ltable_id'][ind]],'left_price':df1['price'][dfTs['ltable_id'][ind]],'right_name':df2['name'][dfTs['rtable_id'][ind]],'right_description':df2['description'][dfTs['rtable_id'][ind]],'right_price':df2['price'][dfTs['rtable_id'][ind]]}, ignore_index=True)

dfVlNew = pd.DataFrame(columns = ['pair_id', 'label','left_name', 'left_description', 'left_price','right_name','right_description','right_price'])
for ind in dfVl.index:
  pair_id=str(dfVl['ltable_id'][ind])+"#"+str(dfVl["rtable_id"][ind])
  dfVlNew= dfVlNew.append({'pair_id': pair_id, 'label': dfVl['label'][ind], 'left_name': df1['name'][dfVl['ltable_id'][ind]],'left_description':df1['description'][dfVl['ltable_id'][ind]],'left_price':df1['price'][dfVl['ltable_id'][ind]],'right_name':df2['name'][dfVl['rtable_id'][ind]],'right_description':df2['description'][dfVl['rtable_id'][ind]],'right_price':df2['price'][dfVl['rtable_id'][ind]]}, ignore_index=True)

dfTrNew.head()

,pair_id,label,left_name,left_description,left_price,right_name,right_description,right_price
0,730#775,0,lg 24 ' lds4821ww semi integrated built in whi...,lg 24 ' lds4821ww semi integrated built in whi...,NaN,lg ldf6920bb fully integrated dishwasher,NaN,NaN
1,670#820,0,speck seethru clear hard shell case for macboo...,speck seethru clear hard shell case for macboo...,NaN,speck products seethru case for apple 13 ' mac...,plastic pink,NaN
2,497#239,0,denon blu-ray disc dvd/cd player dvd3800bdci,denon blu-ray disc dvd/cd player dvd3800bdci 1...,1999.0,denon dvd-2930ci dvd player dvd2930ci,"dvd + rw , dvd-rw , cd-rw dvd video , dvd audi...",448.00
3,644#299,0,panasonic dect 6.0 expandable digital cordless...,panasonic dect 6.0 expandable digital cordless...,NaN,panasonic kx-tg1032s dual handset digital cord...,1 x phone line ( s ) headset jack silver,61.14
4,464#32,0,sony silver minidv handycam camcorder dcrhc52,sony silver minidv handycam camcorder dcrhc52 ...,NaN,sony minidv head cleaner dvm12cld,head cleaner,7.95


In [ ]:
train = dfTrNew
test= dfTsNew
validation= dfVlNew

In [ ]:
train.to_csv(r'/content/drive/My Drive/AbtBuyData/trainNew.csv',index=False)
test.to_csv(r'/content/drive/My Drive/AbtBuyData/testNew.csv',index=False)
validation.to_csv(r'/content/drive/My Drive/AbtBuyData/validationNew.csv',index=False)

In [ ]:
train, validation, test = dm.data.process(
    path=r'/content/drive/My Drive/AbtBuyData',
    train='trainNew.csv',
    validation='validationNew.csv',
    test='testNew.csv',
    id_attr='pair_id',
    label_attr='label',
    left_prefix='left_',
    right_prefix='right_',
    embeddings='fasttext.en.bin')


Reading and processing data from "/content/drive/My Drive/AbtBuyData/trainNew.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/drive/My Drive/AbtBuyData/validationNew.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/drive/My Drive/AbtBuyData/testNew.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


In [ ]:
!ls -rtla /root/.vector_cache

total 8294620
drwx------ 1 root root       4096 Oct  6 10:26 ..
drwxr-xr-x 2 root root       4096 Oct  6 10:26 .
-rw-r--r-- 1 root root 8493673445 Oct  6 10:28 wiki.en.bin


In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=10,
    batch_size=3,
    best_save_path='/content/drive/My Drive/deepmatcherresult/dm_bestpathFt.ph',
    pos_neg_ratio=3)

* Number of trainable parameters: 7133105
===>  TRAIN Epoch 1


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2352: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:03:42


Finished Epoch 1 || Run Time:  209.1 | Load Time:   13.0 || F1:  39.23 | Prec:  44.79 | Rec:  34.90 || Ex/s:  25.85

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:36


Finished Epoch 1 || Run Time:   32.1 | Load Time:    4.4 || F1:  54.22 | Prec:  46.23 | Rec:  65.53 || Ex/s:  52.41

* Best F1: tensor(54.2169, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:03:35


Finished Epoch 10 || Run Time:  202.6 | Load Time:   12.7 || F1:  96.30 | Prec:  95.53 | Rec:  97.08 || Ex/s:  26.67

===>  EVAL Epoch 10


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:35


Finished Epoch 10 || Run Time:   31.4 | Load Time:    4.3 || F1:  61.18 | Prec:  65.03 | Rec:  57.77 || Ex/s:  53.58

---------------------

Loading best model...
Training done.


tensor(65.0943, device='cuda:0')

In [ ]:
model.run_eval(test,batch_size=8, device='cuda:0', progress_style='bar', log_freq=5, sort_in_buckets=None)


===>  EVAL Epoch 6


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:16


Finished Epoch 6 || Run Time:   11.9 | Load Time:    4.6 || F1:  61.35 | Prec:  61.06 | Rec:  61.65 || Ex/s: 115.94



tensor(61.3527, device='cuda:0')

In [ ]:
model.save_state(r'/content/drive/My Drive/DeepMatcherTest/finalmodel.pth')
model.save_state(r'/content/drive/My Drive/DeepMatcherTest/finalmodel.sav')